In [1]:
import pandas as pd
import glob


df = pd.read_json('News_Category_Dataset.json',lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [2]:
df['data'] = df['headline']+str('. ')+df['short_description']
#print(df['data'].head())
df1=df[['data','category']]
print(df1.head())

                                                data       category
0  There Were 2 Mass Shootings In Texas Last Week...          CRIME
1  Will Smith Joins Diplo And Nicky Jam For The 2...  ENTERTAINMENT
2  Hugh Grant Marries For The First Time At Age 5...  ENTERTAINMENT
3  Jim Carrey Blasts 'Castrato' Adam Schiff And D...  ENTERTAINMENT
4  Julianna Margulies Uses Donald Trump Poop Bags...  ENTERTAINMENT


In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df1['category']  =le.fit_transform(df1['category'])
df1.head()

c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,data,category
0,There Were 2 Mass Shootings In Texas Last Week...,6
1,Will Smith Joins Diplo And Nicky Jam For The 2...,8
2,Hugh Grant Marries For The First Time At Age 5...,8
3,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,8
4,Julianna Margulies Uses Donald Trump Poop Bags...,8


In [47]:
print(list(le.classes_))
le.transform(list(le.classes_))

[u'ARTS', u'ARTS & CULTURE', u'BLACK VOICES', u'BUSINESS', u'COLLEGE', u'COMEDY', u'CRIME', u'EDUCATION', u'ENTERTAINMENT', u'FIFTY', u'GOOD NEWS', u'GREEN', u'HEALTHY LIVING', u'IMPACT', u'LATINO VOICES', u'MEDIA', u'PARENTS', u'POLITICS', u'QUEER VOICES', u'RELIGION', u'SCIENCE', u'SPORTS', u'STYLE', u'TASTE', u'TECH', u'THE WORLDPOST', u'TRAVEL', u'WEIRD NEWS', u'WOMEN', u'WORLD NEWS', u'WORLDPOST']


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
      dtype=int64)

In [7]:
training_data = df1
training_data.to_csv("train_data.csv", sep=',', encoding='utf-8')
print(training_data.data.shape)


(124989,)


In [9]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.data)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))

In [45]:
# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

#clf = MultinomialNB().fit(X_train_tfidf, training_data.flag)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.category, test_size=0.25, random_state=42)
clf = MultinomialNB().fit(X_train, y_train)

#SAVE MODEL
pickle.dump(clf, open("nb_model.pkl", "wb"))

In [49]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

docs_new = "Messi killed 3 persons"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)
print(predicted)
# print(category_list[predicted[0]])






[17]


In [68]:
predicted = loaded_model.predict(X_test)
result_bayes = pd.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_bayes.to_csv('res_bayes.csv', sep = ',')
y_test = y_test.astype(int)
count = 0
for predicted_item, result in zip(predicted, y_test):
    #print(predicted_item, ' - ',result)
    if(predicted_item==result):
        count = count + 1
print(len(y_test))
count

31248


11084

In [51]:
from sklearn.metrics import confusion_matrix  

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)

[[   0    0    0    0    0    0    0    0  111    0    0    0    0    0
     0    0    0  273    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0    0    0    0    0  105    0    0    0    0    0
     0    0    0  239    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0    0    0    0    0  236    0    0    0    2    0
     0    0    0  731    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    4    0    0    0    0   22    0    0    0    6    0
     0    0    0 1038    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0    0    0    0    0    4    0    0    0    0    0
     0    0    0  284    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    1    0    0    0    0    0  252    0    0    0    2    0
     0    0    0  758    0    0    0    0    0    1    0    0    0    0
     0    0    0]
 [   0    0    0    0    0  

In [53]:
from sklearn.neural_network import MLPClassifier

clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.category, test_size=0.25, random_state=42)

clf_neural.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [54]:
pickle.dump(clf_neural, open("softmax.pkl", "wb"))

In [69]:
predicted = clf_neural.predict(X_test)
result_softmax = pd.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_softmax.to_csv('res_softmax.csv', sep = ',')
y_test = y_test.astype(int)
count = 0
for predicted_item, result in zip(predicted, y_test):
    #print(predicted_item, ' - ',result)
    if(predicted_item==result):
        count = count + 1
print(len(y_test))
count

31248


16091

In [12]:
from sklearn import svm
from sklearn.model_selection import train_test_split
clf_svm = svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.category, test_size=0.25, random_state=42)
clf_svm.fit(X_train_tfidf, training_data.category)
pickle.dump(clf_svm, open("svm.pkl", "wb"))

In [14]:
predicted = clf_svm.predict(X_test)
result_svm = pd.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_svm.to_csv('res_svm.csv', sep = ',')
y_test = y_test.astype(int)
count = 0
for predicted_item, result in zip(predicted, y_test):
    #print(predicted_item, ' - ',result)
    if(predicted_item==result):
        count = count + 1
print(len(y_test))
print(count)

31248
28739


In [15]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test,predicted)

(array([0.97375328, 0.98442368, 0.9158361 , 0.92721834, 0.94285714,
        0.92638732, 0.94049347, 0.97029703, 0.91009755, 0.95439739,
        0.95224719, 0.92042042, 0.8733674 , 0.93684211, 0.9832636 ,
        0.93465046, 0.89311408, 0.8984339 , 0.9473251 , 0.93610224,
        0.95770393, 0.92746615, 0.96509599, 0.94787645, 0.95737705,
        0.91666667, 0.96383363, 0.96979332, 0.90464241, 0.97430407,
        0.92941176]),
 array([0.96614583, 0.91860465, 0.85345717, 0.86915888, 0.91666667,
        0.80670611, 0.9323741 , 0.86343612, 0.94861226, 0.85174419,
        0.88976378, 0.92319277, 0.92150989, 0.82662539, 0.84229391,
        0.84827586, 0.91473684, 0.96178501, 0.93198381, 0.91419657,
        0.91618497, 0.95517928, 0.94207836, 0.97420635, 0.92113565,
        0.91276596, 0.98158379, 0.92987805, 0.824     , 0.80530973,
        0.87086614]),
 array([0.96993464, 0.95037594, 0.88354701, 0.89725036, 0.92957746,
        0.86241434, 0.93641618, 0.91375291, 0.92895587, 0.90015361,
    

In [18]:
print(predicted)

[17 17 20 ... 17 19 17]
